In [ ]:
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [ ]:
data = pd.read_csv('/composite/clinical.csv',sep=";")

In [ ]:
local_path = '/composite/local'

local_images = []
for i in data.ID.values:
  basePath = os.path.sep.join([local_path, "{}.jpg".format(i)])
  image = cv2.imread(basePath)
  image = cv2.resize(image, (224, 224))
  local_images.append(image)

local_imageData = np.array(local_images)

In [ ]:
whole_path = 'composite/whole'

whole_images = []
for i in data.ID.values:
  basePath = os.path.sep.join([whole_path, "{}.jpg".format(i)])
  image = cv2.imread(basePath)
  image = cv2.resize(image, (224, 224))
  whole_images.append(image)

whole_imageData = np.array(whole_images)

In [ ]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
def create_mlp(dim, regress=False):
	# define a MLP network for clinical data
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))

	return model


In [ ]:
local_net = load_model('composite/local.h5')
whole_net = load_model('composite/whole.h5')

In [ ]:
local_net = Model(inputs=local_net.input, outputs=local_net.get_layer('activation_4').output)
whole_net = Model(inputs=whole_net.input, outputs=whole_net.get_layer('activation_9').output)

In [ ]:
values = ["age", "sex" "R", "U", "mci", "menarche status", "monthes post menarch","initial cobb"]
clin = np.array(data[values])

In [ ]:
mlp = create_mlp(train_clin.shape[1], regress=False)

In [ ]:
Y = data.Type_num.values


split = train_test_split(Y, local_imageData, whole_imageData, clin, test_size=0.2, random_state=4)

In [ ]:
(train_Y, test_Y, train_local, test_local, train_whole, test_whole, train_clin, test_clin) = split

In [ ]:
for layer in cnn_R.layers:
   layer.trainable = False
for layer in cnn_U.layers:
   layer.trainable = False

combinedInput = concatenate([local_net.output, whole_net.output, mlp.output])

x = Dense(16, activation="relu")(combinedInput)
x = Activation("relu")(x)
attention = tf.keras.layers.Attention()([combinedInput, x])
x = Dense(8, activation="relu")(attention)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[local_net.input, whole_net.input, mlp.input], outputs=x)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#opt = Adam(lr=1e-2, decay=1e-3 / 200)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
# train the model
print("[INFO] training model...")
model.fit(
	x=[train_Y, train_local,  train_whole,  train_clin], y=train_Y,
	validation_data=([test_Y, test_local,  test_whole,  test_clin], test_Y),
	epochs=200, batch_size=32)